In [ ]:
"""https://api.edamam.com/search?q=${prompt}&app_id=7aa516a5&app_key=dc836a223fb788b11ae390504d9e97ce&from=0&to=10"""
import torch
import accelerate
from dataclasses import fields,dataclass
import pandas as pd
from typing import Optional
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig,prepare_model_for_kbit_training
from transformers import (AutoTokenizer, 
                        AutoModelForCausalLM,
                        pipeline,
                        BitsAndBytesConfig,
                        GemmaTokenizer,
                        HfArgumentParser,
                        TrainingArguments)

In [ ]:
from huggingface_hub import interpreter_login
interpreter_login()

In [ ]:


# dataset = load_dataset('Amod/mental_health_counseling_conversations',split='train')
# dataset
# df = pd.DataFrame(dataset)
df = pd.read_csv('./modify2.csv')
# 13 is threshold for this dataset
# df = df[:12]
# Split the data into training and evaluation sets

df.head()
df.shape
df

In [ ]:
def format_rows(row):
    q=row['Input']
    a=row['Output']
    # Formatting our dataset to LLaMa format
    format = f'[INST] {q} [/INST] {a}'
    return format

df['formatted'] = df.apply(format_rows,axis=1)

In [ ]:
new_df = df.rename(columns={'formatted':'text'})
new_df = new_df[['text']]
new_df.head()
train_df, eval_df = train_test_split(new_df, test_size=0.26, random_state=42)
print(new_df.shape)
new_df.head()

In [ ]:

# Save the splits to CSV files if needed
train_df.to_csv('./train.csv', index=False)
eval_df.to_csv('./eval.csv', index=False)

training_ds = load_dataset('csv', data_files='./train.csv', split='train')
evaluation_ds = load_dataset('csv', data_files='./eval.csv', split='train')



In [ ]:
# model_id = "microsoft/phi-2"
# new_model = 'Recipe-Generator'
# tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast =True)

# tokenizer.pad_token =tokenizer.eos_token

# tokenizer.padding_side = 'right'
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type='nf4', #normalising float 4
#     bnb_4bit_compute_dtype=torch.float16,
#     # Double quantization can degrade the  performance
#     bnb_4bit_use_double_quant=False
# )
# try:

#     model =AutoModelForCausalLM.from_pretrained(
#                                         model_id,
#                                         trust_remote_code = True,
#                                         quantization_config = bnb_config,
#                                         flash_attn=True,
#                                         flash_rotary = True,
#                                         # flash_dense = True,
#                                         low_cpu_mem_usage=True,
#                                         device_map = {"":0},
#                                         revision='refs/pr/23'
#                                         )

# # Welp try this in runpod.ai bc my CUDA memory is running out
#     model.config.use_cache = False
#     model.config.pretraining_tp = 1
#     model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
#     training_args = TrainingArguments(
#         output_dir = './Recipe-Generator',
#         num_train_epochs=2,
#         per_device_train_batch_size=2,
#         gradient_accumulation_steps=32,
#         eval_strategy = 'steps',
#         eval_steps = 1500,
#         optim = 'paged_adamw_8bit',
#         learning_rate = 2e-4,
#         lr_scheduler_type='cosine',
#         save_steps = 1500,
#         warmup_ratio=0.05,
#         weight_decay=0.01,
#         max_steps=-1
#     )

#     peft_config = LoraConfig(
#         r=32,
#         lora_alpha=64,
#         lora_dropout = 0.05,
#         bias='none',
#         task_type='CAUSAL_LM',
#         target_modules=['Wqkv','fc1','fc2']
#     )

#     trainer = SFTTrainer(
#         model = model,
#         train_dataset=training_ds,
#         eval_dataset = evaluation_ds,
#         peft_config=peft_config,
#         dataset_text_field='text',
#         tokenizer=tokenizer,
#         args=training_args
#     )
#     trainer.train()
#     trainer.save_model('./Recipe-Generator')
#     tokenizer.save_pretrained('./Recipe-Generator')
# except Exception as e:
#     print('At line:',e.__traceback__.tb_lineno)
#     print('________________ERROR________________:',e)


from transformers import IntervalStrategy

model_id = "microsoft/phi-2"
new_model = 'Recipe-Generator'

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

# BitsAndBytes configuration for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',  # normalizing float 4
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False  # Avoid double quantization for better performance
)

try:
    # Load model with quantization
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        quantization_config=bnb_config,
        flash_attn=True,
        flash_rotary=True,
        low_cpu_mem_usage=True,
        device_map={"": 0},
        revision='refs/pr/23'
    )

    # Set model configuration for training
    model.config.use_cache = False
    model.config.pretraining_tp = 1

    # Prepare model for k-bit training
    model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

    # Training arguments
    training_args = TrainingArguments(
        output_dir='./Recipe-Generator',
        num_train_epochs=1,
        per_device_train_batch_size=1,  # Reduce batch size to fit in memory
        gradient_accumulation_steps=64,  # Increase gradient accumulation steps
        eval_strategy=IntervalStrategy.STEPS,
        eval_steps=1500,
        optim='paged_adamw_8bit',
        learning_rate=2e-4,
        lr_scheduler_type='cosine',
        save_steps=1500,
        warmup_ratio=0.05,
        weight_decay=0.01,
        fp16=True,  # Use mixed precision
        max_steps=-1
    )

    # PEFT configuration
    peft_config = LoraConfig(
        r=32,
        lora_alpha=64,
        lora_dropout=0.05,
        bias='none',
        task_type='CAUSAL_LM',
        target_modules=['Wqkv', 'fc1', 'fc2']
    )

    # Initialize trainer
    trainer = SFTTrainer(
        model=model,
        train_dataset=training_ds,
        eval_dataset=evaluation_ds,
        peft_config=peft_config,
        dataset_text_field='text',
        tokenizer=tokenizer,
        args=training_args
    )

    # Train and save the model
    trainer.train()
    trainer.save_model('./Recipe-Generator')
    tokenizer.save_pretrained('./Recipe-Generator')

except Exception as e:
    print('At line:', e.__traceback__.tb_lineno)
    print('________________ERROR________________:', e)


# # Adjust max sequence length in tokenization
# max_sequence_length = 690
# def preprocess_example(example):
#     example['text'] = example['text'][:max_sequence_length]
#     return example
# training_ds = training_ds.map(preprocess_example)

# model_id = "microsoft/phi-2"
# new_model = 'mental-health-LLM'
# tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = 'right'

# try:
#     model = AutoModelForCausalLM.from_pretrained(
#         model_id,
#         trust_remote_code=True,
#         low_cpu_mem_usage=True,
#         device_map={"": 0},
#         revision='refs/pr/23'
#     )

#     model.config.use_cache = False
#     model.config.pretraining_tp = 1

#     training_args = TrainingArguments(
#         output_dir='./mental-healthLLM',
#         num_train_epochs=2,
#         per_device_train_batch_size=2,  # Reduced batch size for T4
#         gradient_accumulation_steps=32,  # Increased to maintain effective batch size
#         evaluation_strategy='steps',
#         eval_steps=1500,
#         optim='paged_adamw_8bit',
#         learning_rate=2e-4,
#         lr_scheduler_type='cosine',
#         save_steps=1500,
#         warmup_ratio=0.05,
#         weight_decay=0.01,
#         max_steps=-1,
#         fp16=True  # Enable mixed precision to save memory
#     )

#     peft_config = LoraConfig(
#         r=32,
#         lora_alpha=64,
#         lora_dropout=0.05,
#         bias='none',
#         task_type='CAUSAL_LM',
#         target_modules=['Wqkv', 'fc1', 'fc2']
#     )

#     trainer = SFTTrainer(
#         model=model,
#         train_dataset=training_ds,
#         peft_config=peft_config,
#         dataset_text_field='text',  # Use the 'formatted' column
#         tokenizer=tokenizer,
#         args=training_args
#     )

#     trainer.train()
#     # Save the fine-tuned model and tokenizer
#     trainer.save_model('./mental-healthLLM')
#     tokenizer.save_pretrained('./mental-healthLLM')

# except Exception as e:
#     print('At line:', e.__traceback__.tb_lineno)
#     print('________________ERROR________________:', e)

In [ ]:
from transformers import (AutoTokenizer, 
                        AutoModelForCausalLM,
                        pipeline,
                        BitsAndBytesConfig,
                        GemmaTokenizer,
                        HfArgumentParser,
                        TrainingArguments)

import torch
import accelerate            
try:
   # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained('./Recipe-Generator')

    # Load the fine-tuned model from the checkpoint
    model = AutoModelForCausalLM.from_pretrained('./Recipe-Generator')

    # Prepare the input text
    input_text = "how to make chicken 65 dish"

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generate text
    output = model.generate(input_ids, max_length=1024, num_return_sequences=1, no_repeat_ngram_size=2)

    # Decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    print(generated_text)
except Exception as e:
    print('At line:', e.__traceback__.tb_lineno)
    print(e)